## **Create dataset for Time-dependent Deep Image Prior (TD-DIP) using 1.5T data**
This code takes the data from the MC-CINE at 1.5T, an create a new dataset wit the used format for the TD-DIP acquitecture

#### Libraries

In [1]:
# basic libraries
import numpy as onp
import jax.numpy as np 
import matplotlib.pyplot as plt  
import os
# bart
from inrmri.bart import bart_acquisition_from_arrays

In [2]:
from inrmri.utils_rdls import preprocess_data, get_csm_format
from inrmri.utils_rdls import plot_angles_and_times_from_xdata, balance_spokes_duplicating
from inrmri.utils_rdls import plot_traj, plot_kdata, plot_csm, plot_zoom_images

In [3]:
from inrmri.utils_rdls import save_frames_as_gif_with_pillow

# **1 .- DATA**

#### **Parameters**

In [4]:
# input folders
base_path             = '/mnt/workspace/datasets/CRUZ_TOM_FREERUN/CINE_tr_resolved/Processed_Data/CINE/'
slice_num_list        = [1,2,3,4,5,6,7,8]
nslices               = 8
volunteer_name_list   = ['DW', 'OJ', 'NF', 'GC', 'CM']
# parameters
nbins                 = 30
# computation folders
bart_files_folder     = '/mnt/workspace/datasets/pulseqCINE/bart_files/'
# output folder
dataset_folder        = 'DATA_1.5T'

In [5]:
if not os.path.exists(bart_files_folder):
    os.makedirs(bart_files_folder)

## **Loop**

In [6]:
for volunteer_name in volunteer_name_list:
    base_folder           = '/mnt/workspace/datasets/pulseqCINE/' + dataset_folder + '/' + volunteer_name + '/'
    train_data_folder     = base_folder + 'traindata/'
    recons_folder         = base_folder + 'recons/'  
    print()
    print('volunteer_name: ' + volunteer_name)
    print('base_folder: ' + base_folder)
    print('train_data_folder: ' + train_data_folder)
    print('recons_folder: ' + recons_folder)
    if not os.path.exists(base_folder):
        os.makedirs(base_folder)
    if not os.path.exists(train_data_folder):
        os.makedirs(train_data_folder)
    if not os.path.exists(recons_folder):
        os.makedirs(recons_folder)
    for slice_num in slice_num_list:
        dataset_name              = 'slice_' + str(slice_num) +'_' + str(nslices) +'_nbins' + str(nbins)
        dataset_recon_folder      = recons_folder + dataset_name + '/'
        path_save                 = train_data_folder + dataset_name + '.npz'
        # load data
        zf     = onp.load(base_path + volunteer_name + '_slice' +  str(slice_num) + '_' +'zf' + '.npy', allow_pickle=True)[:,:,0,:]
        traj   = onp.load(base_path + volunteer_name + '_slice' +  str(slice_num) + '_' +'traj' + '.npy', allow_pickle=True)[0]
        kdata  = onp.load(base_path + volunteer_name + '_slice' +  str(slice_num) + '_' +'kdata' + '.npy', allow_pickle=True)[0]
        fs     = onp.load(base_path + volunteer_name + '_slice' +  str(slice_num) + '_' +'fs' + '.npy', allow_pickle=True)[:,:,0,:]
        dcf    = onp.load(base_path + volunteer_name + '_slice' +  str(slice_num) + '_' +'dcf' + '.npy', allow_pickle=True)[0]
        csm    = onp.load(base_path + volunteer_name + '_slice' +  str(slice_num) + '_' +'csm' + '.npy', allow_pickle=True)[:,:,0,:]
        readout, _, n_coils  = csm.shape
        n_frames             = len(traj)
        # ---   preprocessing   ----
        # trajectory
        for i in range(nbins):
            traj[i] = traj[i]*readout
        traj_aux_list = []
        spokes_per_bin_list = []
        bins_sorted = []
        for i in range(n_frames):
            spokes_per_bin = traj[i].shape[0]//readout
            spokes_per_bin_list.append(spokes_per_bin)
            bins_sorted += [i]*spokes_per_bin
            traj_aux_list.append( np.reshape( traj[i], (spokes_per_bin, readout, 2) ) )
        traj_sorted = np.concat(traj_aux_list, axis=0)
        # coils
        csm = np.transpose(csm, (2, 0, 1))
        # data
        kdata_aux_list = []
        for i in range(n_frames):
            kdata_aux_list.append( kdata[i] )
        kdata_sorted = np.concat(kdata_aux_list, axis=1)
        # ---   TD-DIP Dataset   ----
        # times
        times = onp.array(bins_sorted)/n_frames
        # Preprocess data
        Y_data, X_data, spclim = preprocess_data(times, kdata_sorted, traj_sorted)
        # ---   GRASP and SENSE reconstructions   ----
        # csm
        bac_name_csm = dataset_folder + '_' + volunteer_name + '_' + dataset_name
        csm_computed, hollow_mask_computed, bac_csm = get_csm_format(traj_sorted, kdata_sorted, bart_files_folder, bac_name_csm)
        hollow_mask_computed = np.nan_to_num(hollow_mask_computed, nan=1.0)
        # grasp reconstruction
        ktraj_b, kdata_b, time_b = balance_spokes_duplicating(times, kdata_sorted, traj_sorted)
        bac_name_grasp = dataset_folder + '_' + volunteer_name + '_' + dataset_name + '_grasp_b'
        bac_grasp = bart_acquisition_from_arrays(ktraj_b, kdata_b, bart_files_folder, bac_name_grasp)
        iters = 100
        lagrangian_value = 5.
        lambda_value = 0.01
        grasp_exp_name = bac_name_grasp + str(iters) + '_lagrangian' + str(lagrangian_value).replace(".", "_") + '_lambda' + str(lambda_value).replace(".", "_")
        recon_grasp = bac_grasp.calculate_bart_reco_with_external_csmap(bac_csm, grasp_exp_name, lmbda = lambda_value, lagrangian = lagrangian_value, iters = iters)
        recon_grasp = recon_grasp.reshape((recon_grasp.shape[0],recon_grasp.shape[1],recon_grasp.shape[-1]))
        # sense reconstruction
        bac_name_sense = dataset_folder + '_' + volunteer_name + '_' + dataset_name + '_sense_b'
        recon_sense = bac_grasp.make_sense_reconstruction( bac_csm.csmappath(), bac_name_sense)
        recon_sense = recon_sense.reshape((recon_sense.shape[0],recon_sense.shape[1],recon_sense.shape[-1]))
        # save dataset
        onp.savez(path_save, Y_data=Y_data, X_data=X_data, csm=csm, csm_computed=csm_computed, hollow_mask_computed=hollow_mask_computed, spclim=spclim, recon_grasp=recon_grasp, recon_sense=recon_sense, recon_zf=zf, recon_fs=fs)
        # save recons
        recon_grasp_cropped = recon_grasp[recon_grasp.shape[0]//4:3*recon_grasp.shape[0]//4,recon_grasp.shape[1]//4:3*recon_grasp.shape[1]//4,:]
        recon_sense_cropped = recon_sense[recon_sense.shape[0]//4:3*recon_sense.shape[0]//4,recon_sense.shape[1]//4:3*recon_sense.shape[1]//4,:]
        zf_cropped = zf[zf.shape[0]//4:3*zf.shape[0]//4,zf.shape[1]//4:3*zf.shape[1]//4,:]
        fs_cropped = fs[fs.shape[0]//4:3*fs.shape[0]//4,fs.shape[1]//4:3*fs.shape[1]//4,:]
        save_frames_as_gif_with_pillow(dataset_recon_folder, recon_grasp_cropped / onp.max(onp.abs(recon_grasp_cropped)), filename='GRASP', vmax=1, saturation=0.8, fps=30)
        save_frames_as_gif_with_pillow(dataset_recon_folder, recon_sense_cropped / onp.max(onp.abs(recon_sense_cropped)), filename='SENSE', vmax=1, saturation=0.8, fps=30)
        save_frames_as_gif_with_pillow(dataset_recon_folder, zf_cropped / onp.max(onp.abs(zf_cropped)), filename='ZF', vmax=1, saturation=0.8, fps=30)
        save_frames_as_gif_with_pillow(dataset_recon_folder, fs_cropped / onp.max(onp.abs(fs_cropped)), filename='FS', vmax=1, saturation=0.8, fps=30)


volunteer_name: DW
base_folder: /mnt/workspace/datasets/pulseqCINE/DATA_1.5T/DW/
train_data_folder: /mnt/workspace/datasets/pulseqCINE/DATA_1.5T/DW/traindata/
recons_folder: /mnt/workspace/datasets/pulseqCINE/DATA_1.5T/DW/recons/


2025-04-17 23:51:53.857398: W external/xla/xla/service/gpu/nvptx_compiler.cc:836] The NVIDIA driver's CUDA version is 12.2 which is older than the PTX compiler version (12.6.20). Because the driver is older than the PTX compiler version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


The trajectories are centered
Est. image size: 256x256x1
Done.
Done.
(256, 448, 8) (448, 256, 2)
[  1 256  28   8   1   1   1   1   1   1  30   1   1   1   1   1 ]
[256 256   1   8   1   1   1   1   1   1  30   1   1   1   1   1 ]
TV regularization: 0.010000
Regularization terms: 1, Supporting variables: 0
ADMM
Total Time: 56.884353
[  1 256  28   8   1   1   1   1   1   1  30   1   1   1   1   1 ]
[256 256   1   8   1   1   1   1   1   1  30   1   1   1   1   1 ]
Regularization terms: 0, Supporting variables: 0
conjugate gradients
Total Time: 14.790876
The trajectories are centered
Est. image size: 256x256x1
Done.
Done.
(256, 448, 8) (448, 256, 2)
[  1 256  28   8   1   1   1   1   1   1  30   1   1   1   1   1 ]
[256 256   1   8   1   1   1   1   1   1  30   1   1   1   1   1 ]
TV regularization: 0.010000
Regularization terms: 1, Supporting variables: 0
ADMM
Total Time: 56.629035
[  1 256  28   8   1   1   1   1   1   1  30   1   1   1   1   1 ]
[256 256   1   8   1   1   1   1   1  